<a href="https://colab.research.google.com/github/Coperr/information-retrieval/blob/main/notebook2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# inforet 2

In [ ]:
# no time to lose:
!wget https://gerdes.fr/saclay/inforet/our_msmarco.zip
!unzip our_msmarco.zip
# this will be big: 1.2gb!
# you will get three files

--2025-03-20 08:09:44--  https://gerdes.fr/saclay/inforet/our_msmarco.zip
Resolving gerdes.fr (gerdes.fr)... 54.38.81.127
Connecting to gerdes.fr (gerdes.fr)|54.38.81.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458004951 (437M) [application/zip]
Saving to: ‘our_msmarco.zip’

our_msmarco.zip     100%[===================>] 436.79M  17.5MB/s    in 25s     

2025-03-20 08:10:11 (17.3 MB/s) - ‘our_msmarco.zip’ saved [458004951/458004951]

Archive:  our_msmarco.zip
  inflating: our.msmarco.docs.tsv    
  inflating: our.msmarco.queries.tsv  
  inflating: our.msmarco.gold.tsv    


## ↓ go down to the section "reading in our smaller files"


In [ ]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
# stop using:
# %unload_ext autotime
!pip install sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
time: 1.81 s (started: 2025-03-20 08:11:00 +00:00)


In [ ]:
!pip install scikit-learn
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import dask.dataframe as dd

time: 15.7 s (started: 2025-03-20 08:11:02 +00:00)


## our dataset

- "TREC stands for the Text Retrieval Conference. Started in 1992 it is a series of workshops that focus on supporting research within the information retrieval community. It provides the infrastructure necessary for large-scale evaluation of text retrieval methodologies. Every year these workshops are organized, which are centered around a set of tracks. These tracks encourage new researches in the area of information retrieval."
- TREC 2019 Deep Learning Track https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019
- data from MS-Marco https://microsoft.github.io/msmarco/
- The dataset contains  367k queries and a corpus of 3.2 million documents.
___
- if you want to reproduce my selection or get a bigger set, uncomment and execute


In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
!wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz

!gzip -d msmarco-docs.tsv.gz
!gzip -d msmarco-doctrain-queries.tsv.gz
!gzip -d msmarco-doctrain-top100.gz


--2025-03-20 08:11:17--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 409 Public access is not permitted on this storage account.
2025-03-20 08:11:18 ERROR 409: Public access is not permitted on this storage account..

--2025-03-20 08:11:18--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 409 Public access is not permitted on this storage account.
2025-03-20 08:11:18 ERROR 409: Public access is not permitted on this storage account..

--2025-03-20 08:11:18--  h

## skip to the section "reading in our smaller files"

come back here if you want to understand how to reduce the dataset or you want to build your own dataset

- we have three datasets:
    
    1. the queries: msmarco-doctrain-queries.tsv
    2. the gold: msmarco-doctrain-top100.tsv is a table containing query_id, doc_id and rank
    3. the actual documents: msmarco-docs.tsv 21GB of text! doc_id, url, title, text

In [ ]:
#all_queries=pd.read_table('msmarco-doctrain-queries.tsv',header=None)
#all_queries.columns=['qid','query']
#print('Shape=>',all_queries.shape)
#all_queries.head()

time: 218 µs (started: 2025-03-16 20:04:32 +00:00)


#### reducing the dataset
- here we take 1000 queries.
- if this is too big for your computer, use this code to build smaller version, starting with the already reduced 1000 query set that we've downloaded before


In [ ]:
#our_queries=all_queries.sample(n=1000,random_state=42).reset_index(drop=True)
#print('Shape=>',our_queries.shape)
#our_queries.head()

time: 212 µs (started: 2025-03-16 20:04:32 +00:00)


In [ ]:
#our_queries.to_csv('our.msmarco.queries.tsv',sep='\t')

time: 253 µs (started: 2025-03-16 20:04:32 +00:00)


#### the gold file
- 36m lines!

In [ ]:
#gold_top100=pd.read_table('msmarco-doctrain-top100',delimiter=' ',header=None)
#gold_top100.columns=['qid','Q0','docid','rank','score','runstring']
#print('Shape=>',gold_top100.shape)
#display(gold_top100.head())
# Reducing train_top100 for training
#our_gold_top100=train_top100[gold_top100['qid'].isin(our_queries['qid'].unique())].reset_index(drop=True)
#print('Shape=>',our_gold_top100.shape)
#our_gold_top100.head()

time: 290 µs (started: 2025-03-16 20:04:32 +00:00)


In [ ]:
#our_gold_top100.to_csv('our.msmarco.gold.tsv',sep='\t')

time: 270 µs (started: 2025-03-16 20:04:32 +00:00)


#### the data file

- it's so big that it's smarter to use dask: https://docs.dask.org/en/stable/

In [ ]:
#df=dd.read_table('msmarco-docs.tsv',blocksize=100e6,header=None) #  partitions of 100MB
#df.columns=['docid','url','title','body']
#df.head()

time: 249 µs (started: 2025-03-16 20:04:32 +00:00)


In [ ]:
# can't get the number of rows quickly :s
# very slow:
# len(df.index)

# faster:
#!wc -l msmarco-docs.tsv

time: 303 µs (started: 2025-03-16 20:04:32 +00:00)


- big dataset with 3m rows!
- we want the top 100 for our queries
- this takes some time!

In [ ]:
#def create_corpus(result):
#  unique_docid=result['docid'].unique()
#  condition=df['docid'].isin(unique_docid)
#  corpus=df[condition].reset_index(drop=True)
#  corpus=corpus.drop(columns='url')
#  print('Number of Rows=>',len(corpus))
#  return corpus

#our_docs=create_corpus(our_gold_top100)
#our_docs.head()

time: 304 µs (started: 2025-03-16 20:04:32 +00:00)


In [ ]:
#our_docs.to_csv('our.msmarco.docs.tsv',sep='\t', single_file=True)

time: 273 µs (started: 2025-03-16 20:04:32 +00:00)


- this is still a big file: 92k documents

# reading in our smaller files
here we use the

- !wget https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip
- !unzip our_msmarco.zip

In [ ]:
queries = pd.read_csv('our.msmarco.queries.tsv',sep='\t',usecols=[1,2])
queries

,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...
...,...,...
995,89597,cell voltage mv meaning
996,1167043,what an ip address
997,737304,what is daily max citizens atm
998,156934,do i need a florida commercial driver license


time: 160 ms (started: 2025-03-20 08:11:28 +00:00)


In [ ]:
gold = pd.read_csv('our.msmarco.gold.tsv',sep='\t',usecols=[1,3,4,5])
gold

,qid,docid,rank,score
0,310290,D579750,1,-5.11498
1,310290,D579754,2,-5.57703
2,310290,D2380815,3,-5.84852
3,310290,D822566,4,-5.95002
4,310290,D2249695,5,-6.08326
...,...,...,...,...
99995,257942,D253854,96,-6.32693
99996,257942,D3056621,97,-6.32837
99997,257942,D1323491,98,-6.32871
99998,257942,D2722485,99,-6.33100


time: 511 ms (started: 2025-03-20 08:11:30 +00:00)


In [ ]:
docs = pd.read_csv('our.msmarco.docs.tsv',sep='\t',usecols=[1,2,3])
docs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 32.2 s (started: 2025-03-20 08:11:31 +00:00)


In [ ]:
# Creating Training Set of Queries
training_queries=queries.iloc[:500]
print('Shape=>',training_queries.shape)
display(training_queries.head())
# Creating Testing Set of Queries
testing_queries=queries.iloc[500:]
print('Shape=>',testing_queries.shape)
testing_queries.head()

Shape=> (500, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


Shape=> (500, 2)


,qid,query
500,116364,decree verb definition
501,638813,what does hemiballistic mean
502,401631,is advil considered aspirin?
503,1050265,who sang the song midnight train to georgia
504,632336,what does aq mean in chemistry


time: 31.4 ms (started: 2025-03-20 08:12:03 +00:00)


## exploring the data

### 🚧 todo: check whether there are NaN and take care of them

In [ ]:
print("NaN values in queries:")
print(queries.isna().sum())

print("\nNaN values in gold:")
print(gold.isna().sum())

print("\nNaN values in docs:")
print(docs.isna().sum())

NaN values in queries:
qid      0
query    0
dtype: int64

NaN values in gold:
qid      0
docid    0
rank     0
score    0
dtype: int64

NaN values in docs:
docid      0
title    372
body       3
dtype: int64
time: 53.3 ms (started: 2025-03-20 08:12:03 +00:00)


In [ ]:
docs['title'] = docs['title'].fillna("")
docs['body'] = docs['body'].fillna("")

# verification
print("Remaining NaN values in docs:")
print(docs.isna().sum())

Remaining NaN values in docs:
docid    0
title    0
body     0
dtype: int64
time: 119 ms (started: 2025-03-20 08:12:03 +00:00)


### let's have a look at some random query:

In [ ]:
queries.loc[111]

,111
qid,251898
query,how long does getting a doctorate take


time: 4.74 ms (started: 2025-03-20 08:12:12 +00:00)


In [ ]:
gold[gold.qid==251898]

,qid,docid,rank,score
36200,251898,D2865964,1,-4.74293
36201,251898,D3557816,2,-4.90695
36202,251898,D2723985,3,-4.95911
36203,251898,D1951655,4,-4.97272
36204,251898,D1709749,5,-5.02176
...,...,...,...,...
36295,251898,D2531901,96,-5.56896
36296,251898,D2956542,97,-5.57138
36297,251898,D301873,98,-5.57262
36298,251898,D2952336,99,-5.57504


time: 18.3 ms (started: 2025-03-20 08:12:12 +00:00)


### 🚧 todo: let's look at the top-ranked document for that query
- title
- body

In [ ]:
# todo: .values[0] can help

time: 243 µs (started: 2025-03-20 08:12:14 +00:00)


In [ ]:
top_doc_id = gold[gold["qid"] == 251898].sort_values(by="rank").iloc[0]['docid']
print(f"Top document ID: {top_doc_id}")

top_doc = docs[docs["docid"] == top_doc_id]

print("\nTITLE:")
print(top_doc['title'].values[0])
print("\nBODY:")
print(top_doc['body'].values[0])

Top document ID: D2865964

TITLE:
How long does it take to get a post-doctoral degree?

BODY:
Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Graduate Degrees How long does it take to get a post-doctoral degree? Flag How long does it take to get a post-doctoral degree? Answer by Joe Ragusa Confidence votes 98.6KI am not aware of any such thing as a postdoctoral degree. You may be referring to postdoctoral research which may be required for obtaining a tenure-track faculty position, especially at research oriented institutions. Some have suggested that postdoctoral appointments - that were traditionally optional - have become mandatory as demand for tenure-track positions in academia has drastically increased over previous decades (Wikipedia). The length of time in this case depends on the type, content, scope, and depth of the research.2 people found this useful Was this answer useful? Yes Somewhat No Tyler Durden9988 1,326 Contributions How long does

### 🚧 todo: let's look at the second document
- let's make a functioin to make that easier

In [ ]:
def titleAndBody(qid,rank):
    doc_row = gold[(gold["qid"] == qid) & (gold["rank"] == rank)] # find doc id for a qid and a rank

    if doc_row.empty:
        print("No document found for qid=",qid," and rank=",rank)
        return

    doc_id = doc_row["docid"].values[0]
    doc = docs[docs["docid"] == doc_id]

    if doc.empty:
        print("Document ID ",doc_id," not found in docs dataset")
        return

    title = doc["title"].values[0]
    body = doc["body"].values[0]

    display("TITLE:",title)
    display("BODY:",body)

titleAndBody(251898,1)

'TITLE:'

'How long does it take to get a post-doctoral degree?'

'BODY:'

"Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Graduate Degrees How long does it take to get a post-doctoral degree? Flag How long does it take to get a post-doctoral degree? Answer by Joe Ragusa Confidence votes 98.6KI am not aware of any such thing as a postdoctoral degree. You may be referring to postdoctoral research which may be required for obtaining a tenure-track faculty position, especially at research oriented institutions. Some have suggested that postdoctoral appointments - that were traditionally optional - have become mandatory as demand for tenure-track positions in academia has drastically increased over previous decades (Wikipedia). The length of time in this case depends on the type, content, scope, and depth of the research.2 people found this useful Was this answer useful? Yes Somewhat No Tyler Durden9988 1,326 Contributions How long does it take to get a doctoral degree? It totally depends on the actual degree and school but you

time: 19.5 ms (started: 2025-03-20 08:12:16 +00:00)


#### let's look at the 100th document

In [ ]:
titleAndBody(251898,8)

'TITLE:'

'How long does it take to get a doctoral degree?'

'BODY:'

"Tyler Durden9988 1,326 Contributions How long does it take to get a doctoral degree? It totally depends on the actual degree and school but you can assume between three and five years as a general guideline. Medical degrees may take longer due to the internshi …How long does it take to get a doctorate?after your postgraduation you can register your name in any university where you are like to researching. they will give you a topic according to your preference. when you com …How long does it take to be doctor? Approx. 8 years after high school. Unless you get into medical school straight after high school then only 4/5How long does it take a masters in zoology? It takes on average 2 to 3 years to get it depending on how well your grades are. How long does it take to beomce a Doctor? Answer most doctors it take up to 7years of studying then more for actually hands on. But if you really wanna be a doctor then it won't seem long for you. Can you get a doctorate of science in zoology? Yea

time: 19.9 ms (started: 2025-03-20 08:12:18 +00:00)


### 🚧 todo: try this with a few different queries to get a feel of the quality of the gold

# doing our first baseline retrieval function

- todo:
    - build and fit a binary CountVectorizer on the **titles**
    - play with and understand build_analyzer, build_tokenizer, and transform
    - transform our query 111
        - understand what happens with yet unseen words in the transform process
    - find the docs with the most words in common
    - write an evaluation function computing the top 10 precision p@10
    - apply to our 500 queries


In [ ]:
vectorizer = CountVectorizer(binary=True)
# understand the options:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X = vectorizer.fit_transform(docs["title"])
print('we got',len(vectorizer.get_feature_names_out()),'features, for example',vectorizer.get_feature_names_out()[33333:33339])

we got 39559 features, for example ['sputtering' 'sputters' 'sputum' 'spy' 'spyware' 'sq']
time: 1.19 s (started: 2025-03-20 08:12:19 +00:00)


In [ ]:
queries.loc[111].query

'how long does getting a doctorate take'

time: 2.33 ms (started: 2025-03-20 08:12:21 +00:00)


In [ ]:
vectorizer.build_analyzer()(queries.loc[111].query)

['how', 'long', 'does', 'getting', 'doctorate', 'take']

time: 5.37 ms (started: 2025-03-20 08:12:22 +00:00)


In [ ]:
vectorizer.build_tokenizer()(queries.loc[111].query)

['how', 'long', 'does', 'getting', 'doctorate', 'take']

time: 2.85 ms (started: 2025-03-20 08:12:23 +00:00)


In [ ]:
qv = vectorizer.transform([queries.loc[111].query])
qv

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (1, 39559)>

time: 3.12 ms (started: 2025-03-20 08:12:24 +00:00)


### 🚧 todo:
- understand what happens with yet unseen words in the transform process


- think of the shape of X, what are the rows, what are the columns?
- how to select the titles that have the words of our query?
       - think of matrix multiplication and transposition

In [ ]:
print("Shape of X:", X.shape)
print("Number of documents:", docs.shape[0])
print("Number of features:", len(vectorizer.get_feature_names_out()))

query_words = vectorizer.build_analyzer()(queries.loc[111].query)
print("\nQuery words:", query_words)

query_words_in_vocab = [word for word in query_words if word in vectorizer.vocabulary_]
print("Query words in vocabulary:", query_words_in_vocab)

query_words_not_in_vocab = [word for word in query_words if word not in vectorizer.vocabulary_]
print("Query words not in vocabulary:", query_words_not_in_vocab)

#dot product between query vector and document vectors which gives the number of matching terms between query and each document
xqv = X.dot(qv.T)
xqv

Shape of X: (92565, 39559)
Number of documents: 92565
Number of features: 39559

Query words: ['how', 'long', 'does', 'getting', 'doctorate', 'take']
Query words in vocabulary: ['how', 'long', 'does', 'getting', 'doctorate', 'take']
Query words not in vocabulary: []


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 12144 stored elements and shape (92565, 1)>

time: 55.8 ms (started: 2025-03-20 08:12:27 +00:00)


### 🚧 todo:
  - look at argmax and max,
  - check the numpy
      - flatnonzero function to find the best match
      - the .A and the .flat functions
  - show the best matching doc

In [ ]:
xqv_array = xqv.toarray().flatten()

time: 1.28 ms (started: 2025-03-20 08:12:29 +00:00)


In [ ]:
matching_docs_idx = np.flatnonzero(xqv_array)
print("\nNumber of documents with any matching terms:", len(matching_docs_idx))


Number of documents with any matching terms: 12144
time: 3.27 ms (started: 2025-03-20 08:20:48 +00:00)


In [ ]:
best_match_idx = np.argmax(xqv_array)
print("\nBest matching document index:", best_match_idx)
print("Number of matching terms:", xqv_array[best_match_idx])

# best matching document
best_match_doc = docs.iloc[best_match_idx]
print("\nBest matching document:")
print("Title:", best_match_doc['title'])
print("DocID:", best_match_doc['docid'])


Best matching document index: 11957
Number of matching terms: 6

Best matching document:
Title: After getting my bachelors how long does it take to get a doctorate?
DocID: D2408424
time: 2.76 ms (started: 2025-03-20 08:21:07 +00:00)


### 🚧 todo: use argpartition to get the 10 best answers

In [ ]:
pred10i = np.argpartition(-xqv_array, 10)[:10]
pred10i = pred10i[np.argsort(-xqv_array[pred10i])]
pred10i

array([11957, 43190, 91275, 66313, 83787,  3686, 76873, 48618, 76731,
       48830])

time: 8.2 ms (started: 2025-03-20 08:12:32 +00:00)


In [ ]:
docs.loc[pred10i]

,docid,title,body
11957,D2408424,After getting my bachelors how long does it ta...,Education & Reference Higher Education (Univer...
43190,D700799,Holidays By Country Â»Holidays By Religion Â»O...,Home > Important Days > Daylight Savings > Day...
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,Home » College How Long Does It Take to Get a ...
66313,D2956542,How long does it take to finish a doctorate?,Education & Reference Higher Education (Univer...
83787,D1951655,How long does it take to get a Doctorate of Nu...,Answers.com ® Wiki Answers ® Categories Jobs &...
3686,D2111066,After getting a flu shot how long does it take...,Celticballa568 19 Contributions After getting ...
76873,D1444724,Top 20 Doctor insights on: How Long Does Antib...,Top 20 Doctor insights on: How Long Does Antib...
48618,D467553,Naproxen - how long does it take for this drug...,Home › Q & A › Questions › Naproxen - how long...
76731,D970533,How long does it take a Killdeer egg to hatch?,Answers.com ® Wiki Answers ® Categories Animal...
48830,D1829756,How Long Does It Take to Sell a House?,Home Selling How Long Does It Take to Sell a H...


time: 20.2 ms (started: 2025-03-20 08:12:37 +00:00)


In [ ]:
docs.loc[pred10i].docid

,docid
11957,D2408424
43190,D700799
91275,D3090762
66313,D2956542
83787,D1951655
3686,D2111066
76873,D1444724
48618,D467553
76731,D970533
48830,D1829756


time: 6.4 ms (started: 2025-03-20 08:12:38 +00:00)


In [ ]:
gold[gold.qid==251898].docid

,docid
36200,D2865964
36201,D3557816
36202,D2723985
36203,D1951655
36204,D1709749
...,...
36295,D2531901
36296,D2956542
36297,D301873
36298,D2952336


time: 8.01 ms (started: 2025-03-20 08:12:38 +00:00)


### 🚧 todo:
- find the relevant documents that are in our top 10
- user intersect1d
- compute the precision p@10

In [ ]:
actual = gold[gold.qid==251898].docid
found = docs.loc[pred10i].docid
intersection = np.intersect1d(found, actual)
intersection

array(['D1951655', 'D2408424', 'D2956542', 'D3090762'], dtype=object)

time: 9.34 ms (started: 2025-03-20 08:22:30 +00:00)


In [ ]:
precision = len(intersection) / min(10, len(found))
precision

0.4

time: 2.2 ms (started: 2025-03-20 08:12:40 +00:00)


In [ ]:
# 🚧 todo: build a function p@10 that gives the precision at 10
def pAt10(qid):

    query_text = queries[queries.qid == qid]["query"].values[0]
    qv = vectorizer.transform([query_text])

    xqv = (X.dot(qv.T)).toarray().flatten()
    top_10_indices = np.argpartition(-xqv, 10)[:10]  # top 10 document indices
    found_docs = set(docs.iloc[top_10_indices]["docid"])  # top 10 doc IDs
    gold_docs = set(gold[gold.qid == qid]["docid"])

    intersection = np.intersect1d(list(found_docs), list(gold_docs))
    precision = len(intersection) / 10

    return precision

pAt10(251898)


0.4

time: 17 ms (started: 2025-03-20 08:22:48 +00:00)


### 🚧 todo:
- take our 500 training queries qid
- apply our function
- compute the average

In [ ]:
training_queries.qid

,qid
0,687888
1,480210
2,591004
3,260536
4,39422
...,...
495,133970
496,79788
497,791583
498,732078


time: 5.46 ms (started: 2025-03-20 08:12:43 +00:00)


In [ ]:
training_precision = training_queries.qid.apply(pAt10)
training_precision

,qid
0,0.0
1,0.5
2,0.2
3,0.5
4,0.2
...,...
495,0.6
496,0.2
497,0.0
498,0.1


time: 4.81 s (started: 2025-03-20 08:12:44 +00:00)


In [ ]:
training_precision.mean()

np.float64(0.319)

time: 4.34 ms (started: 2025-03-20 08:12:49 +00:00)


- that looks like a baseline we can beat :)
- what's the query we are doing best in?
    - max?

In [ ]:
training_precision.max()
best_query_id = training_queries.qid.iloc[training_precision.idxmax()]
print("The id of the query that we are doing best in : ", best_query_id, " with precision : ", training_precision.max())

The id of the query that we are doing best in :  127412  with precision :  1.0
time: 1.43 ms (started: 2025-03-20 08:12:51 +00:00)


- oh, we have just been lucky before...

## 🚧 todo:

- redo the vectorization and evaluation on the whole text, not only the titles
- try the non-binary CountVectorizer
- go for tf-idf
    - play with at least two options and re-evaluate
- find other improvements. these may include:
    - cleaning the text
    - heuristically combining title and body matches
    - looking at bigrams
    - looking at terms (by means of a clean multi-word term list from wikipedia, see notebook 1)
    - by removing stopwords (look at nltk or spacy to do that)
    - trying an implementation of bm25 (slow with standard implementation, see last cell if you want to do it)
  
- do a grid search with a few promising parameters
    - maybe get inspired by GridSearchCV and pipelines in https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py
        - you can also check the weel-written section "Pipelines" in this book: https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html
    - make a nice visualization of the results
    
- interpret the complete results in 3 to 5 sentences.
    - what strategy would do best if we switch our evaluation to p@100?

- give some ideas for improving the results





IMPORTANT : I WASNT ABLE TO RUN THE TWO LAST CELLS (GRID SEARCH AND PLOT DUE LIMITED RAM RESOURCES AND CRASHES) BUT KEPT THE CODE <br>
ANSWER : <br>
If we add stopwords filters, the p@10 decreases (from 0.045 to 0.33).<br>
CountVectorizer with binary=True performs little better but overall Tf-idf significantly outperforms basic CountVectorizer, with an 88.74% precision@10 vs only 4.48%. The gap is also big for precision@100 (56.26% vs 3.04%)<br>
This gap is due to Tf-idf using downweights on common words and valuing important terms

Also, Using bigrams improves precision, which means that multi-word phrases matter <br>
If we lower max_df, it will probably help in capturing more relevant documents by including more common terms but if we increase min_df, it might help in filtering out very rare terms that could add noise


In [ ]:
# Function for leaning text
# remove words with numbers inside
# replace new lines by space
# remove urls
# only keep ascii words

def clean_text(text):
    text=re.sub('\w*\d+\w*','', text)
    text = re.sub(r'\s+', ' ', text)  # new lines with space
    text = re.sub(r'http\S+', '', text)  # URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # ASCII words
    text = text.lower()  # lowercase
    return text

# Cleaning corpus using RegEx
docs['cleaned']=docs['body'].apply(lambda x: clean_text(x))


time: 7min 39s (started: 2025-03-20 08:13:09 +00:00)


In [ ]:
vectorizer_binary = CountVectorizer(binary=True)
X_binary = vectorizer_binary.fit_transform(docs["cleaned"])

time: 2min 44s (started: 2025-03-16 20:12:22 +00:00)


In [ ]:
vectorizer_count = CountVectorizer(binary=False)
X_count = vectorizer_count.fit_transform(docs["cleaned"])

time: 2min 54s (started: 2025-03-16 20:15:07 +00:00)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(docs["cleaned"])

time: 2min 58s (started: 2025-03-16 20:18:02 +00:00)


In [ ]:
def precision_at_k(qid, vectorizer, X, k):
    #compute precision for a given qid and a vectorizer
    query_text = queries[queries.qid == qid]["query"].values[0]
    qv = vectorizer.transform([query_text])

    xqv = (X.dot(qv.T)).toarray().flatten()
    top_k_indices = np.argpartition(-xqv, k)[:k]
    found_docs = set(docs.iloc[top_k_indices]["docid"])
    gold_docs = set(gold[gold.qid == qid]["docid"])

    intersection = np.intersect1d(list(found_docs), list(gold_docs))
    precision = len(intersection) / k
    return precision

time: 879 µs (started: 2025-03-16 20:21:00 +00:00)


In [ ]:
precision_10_count = training_queries.qid.apply(lambda qid: precision_at_k(qid, vectorizer_count, X_count, k=10))
precision_10_tfidf = training_queries.qid.apply(lambda qid: precision_at_k(qid, vectorizer_tfidf, X_tfidf, k=10))

precision_100_count = training_queries.qid.apply(lambda qid: precision_at_k(qid, vectorizer_count, X_count, k=100))
precision_100_tfidf = training_queries.qid.apply(lambda qid: precision_at_k(qid, vectorizer_tfidf, X_tfidf, k=100))

print("Average p@10 - CountVectorizer:", precision_10_count.mean())
print("Average p@10 - TF-IDF:", precision_10_tfidf.mean())
print("Average p@100 - CountVectorizer:", precision_100_count.mean())
print("Average p@100 - TF-IDF:", precision_100_tfidf.mean())


Average p@10 - CountVectorizer: 0.044800000000000006
Average p@10 - TF-IDF: 0.8874000000000001
Average p@100 - CountVectorizer: 0.030440000000000005
Average p@100 - TF-IDF: 0.56256
time: 17min 15s (started: 2025-03-16 20:21:00 +00:00)


IMPORTANT : I WASNT ABLE TO RUN THE CELLS BELOW (GRID SEARCH AND PLOT DUE LIMITED RAM RESOURCES AND CRASHES)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
])

# parameter grid
parameters = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__min_df': [1, 2, 3],
}

grid_search = GridSearchCV(pipeline, parameters, cv=2, scoring='accuracy', verbose=1)
grid_search.fit(docs['cleaned'], docs['docid'])

print("Best parameters found: ", grid_search.best_params_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# DataFrame the results
results = pd.DataFrame(grid_search.cv_results_)

heatmap_data = results.pivot_table(index='param_tfidf__ngram_range', columns='param_tfidf__max_df', values='mean_test_score')

# plot
plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt=".3f", cmap="YlGnBu")
plt.title('Grid Search Results')
plt.xlabel('max_df')
plt.ylabel('ngram_range')
plt.show()

In [ ]:
import gc
from sklearn.base import BaseEstimator, ClassifierMixin
class BM25Score(BaseEstimator, ClassifierMixin):
    def init(self, vectorized_docs, k1=1.5, b=0.75): # vectorized_docs: word counts
        self.k1 = k1
        self.b = b
        self.vectorized_docs = vectorized_docs
    def fit(self, vectorized_queries=None, query_ids=None, args=None): # args are for grid search integration
        self.n_d = self.vectorized_docs.sum(axis=1).reshape(-1, 1).A
        self.avgdl = np.mean(self.n_d)
        self.n_docs = self.vectorized_docs.shape[0]

        self.nq = np.sum(self.vectorized_docs > 0, axis=0).reshape(1, -1)
        self.idf = np.log(((self.n_docs - self.nq + 0.5) / (self.nq + 0.5)) + 1).A
        return self
    def predict(self, vectorized_queries):
        # bigger batches => faster but more memory heavy
        n_queries = vectorized_queries.shape[0]
        batch_size = 50

        final_scores = []
        for batch_idx in range(int(n_queries / batch_size) + 1):
            start_idx = batch_idx * batch_size
            end_idx = min((batch_idx + 1) * batch_size, n_queries)
            if start_idx >= end_idx:
                break
            batch_vectorized_queries = vectorized_queries[start_idx: end_idx]
            idx_tokens = np.argwhere(batch_vectorized_queries.sum(axis=0) > 0).reshape(-1)
            batch_vectorized_queries = batch_vectorized_queries[:, idx_tokens]
            vectorized_docs = self.vectorized_docs[:, idx_tokens].toarray()
            idf = self.idf[:, idx_tokens]

            scores = idf * (
                (vectorized_docs * (self.k1 + 1)) / \
                (
                    vectorized_docs + \
                    self.k1 * (1 - self.b + self.b * (self.n_d / self.avgdl))
                )
            )
            final_scores.append((batch_vectorized_queries > 0).astype(np.int8) @ scores.T)

            del vectorized_docs
            del scores
            gc.collect()
        return np.concatenate(final_scores, axis=0)

time: 2.14 ms (started: 2025-03-16 20:38:16 +00:00)
